<a href="https://colab.research.google.com/github/yu072333/thursday_fiinal_project/blob/main/%E6%9C%9F%E6%9C%AB%E5%B0%88%E6%A1%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install -q gradio requests google-generativeai gspread pandas

In [27]:
import os
import requests
import google.generativeai as genai
import gradio as gr
import pandas as pd
import gspread
import random
from google.colab import auth
from google.auth import default
from google.colab import userdata
from datetime import datetime

In [28]:
TMDB_API_KEY = userdata.get('tmdb')
GEMINI_API_KEY = userdata.get('gemini')

In [29]:
genai.configure(api_key=GEMINI_API_KEY)

In [30]:
# 正確：SHEET_ID 要用引號包起來，因為它是字串
SHEET_ID = "18rT0H_T6YIXXnzqdypWrF5SfGpNNPvdwvCHF5XPkP6g"

# 授權並開啟 Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
sh = gc.open_by_key(SHEET_ID)
worksheet = sh.sheet1

# 測試是否成功開啟（可選）
print(f"成功開啟試算表：{sh.title}")
print(f"目前工作表名稱：{worksheet.title}")

成功開啟試算表：movie
目前工作表名稱：工作表1


In [31]:
def initialize_sheet():
    """初始化 Sheet 的欄位標題，新增使用者互動欄位"""
    headers = [
        'Timestamp', 'Mood', 'Genre', 'Movie_Name', 'Title', 'Overview',
        'Rating', 'Poster_URL', 'TMDB_ID', 'Source',
        'Favorite', 'My_Rating', 'My_Comment'  # ✨ 新增欄位
    ]

    # 檢查第一列是否為空，如果是則寫入標題
    if worksheet.row_count == 1 or worksheet.cell(1, 1).value is None:
        worksheet.update('A1:M1', [headers])
        print("✅ 已初始化 Google Sheet 資料庫結構 (含新欄位)")
    else:
        # 檢查是否需要補上新欄位 (簡單檢查 K1 是否為 Favorite)
        val = worksheet.acell('K1').value
        if val != 'Favorite':
            worksheet.update('K1:M1', [['Favorite', 'My_Rating', 'My_Comment']])
            print("✅ 已更新舊資料庫結構，補上新欄位")

initialize_sheet()

In [32]:
def load_database():
    """載入資料並整理格式"""
    expected_columns = [
        'Timestamp', 'Mood', 'Genre', 'Movie_Name', 'Title', 'Overview',
        'Rating', 'Poster_URL', 'TMDB_ID', 'Source',
        'Favorite', 'My_Rating', 'My_Comment'
    ]
    try:
        records = worksheet.get_all_records()
        if records:
            df = pd.DataFrame(records)
            # 確保所有欄位存在
            for col in expected_columns:
                if col not in df.columns:
                    df[col] = ""

            # 數值轉型
            df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0)
            df['My_Rating'] = pd.to_numeric(df['My_Rating'], errors='coerce').fillna(0)

            # 字串處理 (Favorite 轉成字串比較好處理)
            df['Favorite'] = df['Favorite'].astype(str)
            return df
        else:
            return pd.DataFrame(columns=expected_columns)
    except Exception as e:
        print(f"讀取資料庫錯誤: {e}")
        return pd.DataFrame(columns=expected_columns)

# 全域變數：暫存資料庫
movie_database = load_database()

In [33]:
def save_to_database(mood, genre, movie_name, movie_details):
    """寫入新電影"""
    if not movie_details: return False

    # 檢查是否已存在 (避免重複)
    global movie_database
    if not movie_database.empty and movie_name in movie_database['Title'].values:
        return True

    row_data = [
        datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        mood, genre, movie_name,
        movie_details['title'], movie_details['overview'], movie_details['rating'],
        movie_details['poster_url'], movie_details['tmdb_id'], 'TMDB',
        "FALSE", 0, "" # 預設空值
    ]

    try:
        worksheet.append_row(row_data)
        # 更新本地緩存
        movie_database = load_database()
        return True
    except Exception as e:
        print(f"寫入失敗: {e}")
        return False

In [34]:
def fetch_movie_details(movie_name):
    """TMDB API 查詢"""
    if not movie_name or not TMDB_API_KEY: return None
    url = "https://api.themoviedb.org/3/search/movie"
    params = {"api_key": TMDB_API_KEY, "query": movie_name, "language": "zh-TW"}
    try:
        resp = requests.get(url, params=params, timeout=5)
        if resp.status_code == 200:
            results = resp.json().get('results', [])
            if results:
                movie = results[0]
                return {
                    'title': movie.get('title'),
                    'overview': movie.get('overview', '無簡介'),
                    'rating': movie.get('vote_average', 0),
                    'poster_url': f"https://image.tmdb.org/t/p/w500{movie['poster_path']}" if movie.get('poster_path') else '',
                    'tmdb_id': movie.get('id')
                }
    except:
        pass
    return None

In [35]:
def recommend_movies(mood, genre):
    """AI 推薦邏輯 (混合 資料庫 + TMDB)"""
    if not mood or not genre: return "請輸入心情與類型"

    prompt = f"推薦 3 部適合心情「{mood}」且類型是「{genre}」的電影。只回傳電影名稱，用英文逗號分隔。"
    output = f"### 🎬 給「{mood}」的你推薦：\n\n"

    try:
        model = genai.GenerativeModel('gemini-2.0-flash')
        response = model.generate_content(prompt)
        names = [n.strip() for n in response.text.split(',') if n.strip()]

        for name in names[:3]:
            # 1. 先查資料庫
            cached = movie_database[movie_database['Title'].str.contains(name, case=False, na=False)]

            if not cached.empty:
                row = cached.iloc[0]
                title = row['Title']
                overview = row['Overview'][:100] + "..."
                rating = row['Rating']
                poster = row['Poster_URL']
                is_fav = "❤️ 已收藏" if str(row['Favorite']) == "TRUE" else ""
                output += f"**{title}** {is_fav}\n(資料庫評分: {rating})\n> {overview}\n\n"
                if poster: output += f"![海報]({poster})\n\n---\n"

            # 2. 資料庫沒有，查 API 並存入
            else:
                details = fetch_movie_details(name)
                if details:
                    save_to_database(mood, genre, name, details)
                    output += f"**{details['title']}** (✨新發現)\n(TMDB: {details['rating']})\n> {details['overview'][:100]}...\n\n"
                    if details['poster_url']: output += f"![海報]({details['poster_url']})\n\n---\n"
                else:
                    output += f"**{name}**\n(暫無詳細資料)\n\n---\n"

        output += "\n💡 **喜歡這些電影嗎？記得到「評分/收藏」分頁加入最愛！**"
        return output
    except Exception as e:
        return f"發生錯誤: {e}"

In [36]:
def feeling_lucky():
    """隨機推薦"""
    global movie_database
    if movie_database.empty: return "❌ 資料庫是空的，先去推薦幾部吧！"

    # 篩選 6 分以上的電影
    candidates = movie_database[movie_database['Rating'] >= 6]
    if candidates.empty: candidates = movie_database

    picked = candidates.sample(1).iloc[0]

    output = f"# 🍀 今天的幸運電影：{picked['Title']}\n"
    output += f"**類型**: {picked['Genre']} | **評分**: {picked['Rating']} ⭐\n"

    if str(picked['Favorite']) == "TRUE":
        output += "**❤️ 這是你的珍藏愛片！**\n"

    output += f"\n> {picked['Overview']}\n\n"
    if picked['Poster_URL']: output += f"![海報]({picked['Poster_URL']})"
    return output


In [37]:
def update_feedback(movie_title, is_favorite, my_rating, my_comment):
    """更新使用者評分與收藏"""
    if not movie_title: return "❌ 未選擇電影"

    try:
        cell = worksheet.find(movie_title)
        row_idx = cell.row

        # 更新 K, L, M 欄 (11, 12, 13)
        worksheet.update_cell(row_idx, 11, "TRUE" if is_favorite else "FALSE")
        worksheet.update_cell(row_idx, 12, my_rating)
        worksheet.update_cell(row_idx, 13, my_comment)

        # 同步更新全域變數，這樣「我的片單」才會即時更新
        global movie_database
        movie_database = load_database()

        return f"✅ 更新成功！\n電影：{movie_title}\n收藏：{'是' if is_favorite else '否'}\n評分：{my_rating}"
    except Exception as e:
        return f"更新失敗: {e}"

In [38]:
def get_titles():
    """下拉選單用的標題列表"""
    if movie_database.empty: return []
    return sorted(movie_database['Title'].astype(str).unique().tolist())

In [39]:
def get_my_list():
    """回傳給 Dataframe 顯示用的資料"""
    if movie_database.empty: return pd.DataFrame()
    # 只顯示重要欄位
    cols = ['Title', 'Rating', 'Favorite', 'My_Rating', 'My_Comment', 'Genre', 'Mood']
    return movie_database[cols]

In [ ]:
with gr.Blocks(title="觀影手帳 Pro", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎬 觀影手帳 Pro: 推薦 + 收藏系統")

    with gr.Tabs():
        # --- Tab 1: AI 推薦 ---
        with gr.TabItem("🤖 AI 智能推薦"):
            with gr.Row():
                mood_in = gr.Textbox(label="現在的心情", placeholder="例如：想大哭一場、熱血沸騰")
                genre_in = gr.Textbox(label="喜歡的類型", placeholder="例如：愛情、科幻")
            btn_rec = gr.Button("🚀 開始推薦", variant="primary")
            out_rec = gr.Markdown()
            btn_rec.click(recommend_movies, [mood_in, genre_in], out_rec)

        # --- Tab 2: 隨機推薦 ---
        with gr.TabItem("🍀 手氣不錯"):
            gr.Markdown("### 不知道看什麼？讓資料庫幫你選！")
            btn_lucky = gr.Button("🎲 隨機抽一部好片", size="lg")
            out_lucky = gr.Markdown()
            btn_lucky.click(feeling_lucky, outputs=out_lucky)

        # --- Tab 3: 評分與收藏 (串接功能核心) ---
        with gr.TabItem("⭐ 評分與收藏"):
            gr.Markdown("### 📝 在這裡記錄你的觀影心得")
            with gr.Row():
                # 下拉選單 (加入 interactive=True)
                movie_dropdown = gr.Dropdown(choices=get_titles(), label="選擇電影 (若找不到請按刷新)", interactive=True, allow_custom_value=False)
                refresh_btn = gr.Button("🔄 刷新電影列表")

            with gr.Group():
                is_fav = gr.Checkbox(label="❤️ 加入我的收藏 (Favorite)")
                my_score = gr.Slider(0, 10, step=0.5, label="⭐ 我的評分", value=0)
                my_comment = gr.Textbox(label="📝 我的短評", lines=2)
                save_btn = gr.Button("💾 儲存評價", variant="primary")

            status_output = gr.Markdown()

            # 按鈕事件
            def refresh_data():
                global movie_database
                movie_database = load_database()
                return gr.Dropdown(choices=get_titles())

            refresh_btn.click(refresh_data, outputs=movie_dropdown)

            save_btn.click(
                update_feedback,
                inputs=[movie_dropdown, is_fav, my_score, my_comment],
                outputs=status_output
            )

        # --- Tab 4: 我的片單 (新增功能) ---
        with gr.TabItem("📊 我的片單總覽"):
            gr.Markdown("### 資料庫中的所有電影與你的評價")
            refresh_table_btn = gr.Button("🔄 更新表格")
            # 使用 Dataframe 元件直接顯示資料
            db_view = gr.Dataframe(value=get_my_list(), interactive=False)

            def refresh_table():
                global movie_database
                movie_database = load_database()
                return get_my_list()

            refresh_table_btn.click(refresh_table, outputs=db_view)

demo.launch(debug=True)

/tmp/ipython-input-442135327.py:1: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="觀影手帳 Pro", theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://53fb419bd8cf438535.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
